# 🔍 X(Twitter) 凍結アカウントチェッカー

Googleスプレッドシートのアカウント一覧をチェックして、凍結・存在しないアカウントを自動で色付けします。

---

## 📋 使い方（3ステップ）

### 1️⃣ 初回のみ：設定を入力
下の「⚙️ 設定」セルでスプレッドシートのURLとシート名を入力してください。

### 2️⃣ 「▶️ すべて実行」をクリック
上のメニューから「ランタイム」→「すべてのセルを実行」を選択します。

### 3️⃣ Googleアカウントで認証
初回のみGoogleアカウントへのアクセス許可が求められます。「許可」をクリックしてください。

---

**⏱️ 所要時間の目安**: 100件で約3〜5分

## ⚙️ 設定（ここを編集してください）

In [ ]:
#@title 📝 スプレッドシートの設定 { display-mode: "form" }

#@markdown ### スプレッドシートのURL
#@markdown スプレッドシートを開いた時のURLをコピペしてください
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/xxxxxxxx/edit" #@param {type:"string"}

#@markdown ### シート名
#@markdown チェックしたいシートの名前（下のタブに表示されている名前）
SHEET_NAME = "シート1" #@param {type:"string"}

#@markdown ### アカウントIDが入っている列
#@markdown A列なら「A」、B列なら「B」と入力
ACCOUNT_COLUMN = "A" #@param {type:"string"}

#@markdown ### ヘッダー行をスキップする
#@markdown 1行目がタイトル行の場合はチェック
SKIP_HEADER = True #@param {type:"boolean"}

print("✅ 設定完了！次のセルに進んでください。")

## 🔧 準備（自動実行されます）

In [ ]:
#@title 📦 必要なライブラリをインストール { display-mode: "form" }
print("📦 ライブラリをインストール中...")
!pip install -q gspread google-auth selenium webdriver-manager
!apt-get update -qq
!apt-get install -qq chromium-browser chromium-chromedriver
print("✅ インストール完了！")

In [ ]:
#@title 🔐 Googleアカウントに接続 { display-mode: "form" }
from google.colab import auth
import gspread
from google.auth import default

print("🔐 Googleアカウントに接続中...")
print("　 ポップアップが表示されたら「許可」をクリックしてください")

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

print("✅ 接続完了！")

In [ ]:
#@title 📊 スプレッドシートを読み込み { display-mode: "form" }
import re

# スプレッドシートIDを抽出
match = re.search(r'/d/([a-zA-Z0-9-_]+)', SPREADSHEET_URL)
if not match:
    raise ValueError("❌ スプレッドシートのURLが正しくありません")

spreadsheet_id = match.group(1)
print(f"📊 スプレッドシートを開いています...")

spreadsheet = gc.open_by_key(spreadsheet_id)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# アカウント一覧を取得
col_num = ord(ACCOUNT_COLUMN.upper()) - ord('A') + 1
all_values = worksheet.col_values(col_num)

# ヘッダーをスキップ
start_row = 2 if SKIP_HEADER else 1
accounts = [(i + start_row, v.strip()) for i, v in enumerate(all_values[start_row - 1:]) if v.strip()]

print(f"✅ {len(accounts)} 件のアカウントを読み込みました")
print(f"　 例: {accounts[0][1] if accounts else 'なし'}")

## 🚀 チェック実行

In [ ]:
#@title ▶️ 凍結チェックを開始 { display-mode: "form" }
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from datetime import datetime
import time
import concurrent.futures

def create_driver():
    """Chromeドライバーを作成"""
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1280,720')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
    
    service = Service('/usr/bin/chromedriver')
    return webdriver.Chrome(service=service, options=options)

def check_account(account_info):
    """単一アカウントをチェック"""
    row, account_name = account_info
    clean_name = account_name.lstrip('@').strip()
    url = f"https://x.com/{clean_name}"
    
    driver = None
    try:
        driver = create_driver()
        driver.set_page_load_timeout(20)
        driver.get(url)
        time.sleep(2)  # JSレンダリング待ち
        
        content = driver.page_source
        
        if 'Account suspended' in content or 'アカウントは凍結されています' in content:
            return (row, clean_name, 'frozen', '凍結')
        elif "This account doesn't exist" in content or 'このアカウントは存在しません' in content:
            return (row, clean_name, 'not_exist', '存在しない')
        elif "Hmm...this page doesn't exist" in content:
            return (row, clean_name, 'not_exist', '存在しない')
        elif 'data-testid="UserName"' in content or ('Following' in content and 'Followers' in content):
            return (row, clean_name, 'normal', '正常')
        else:
            return (row, clean_name, 'unknown', '判定不能')
    except Exception as e:
        return (row, clean_name, 'error', 'エラー')
    finally:
        if driver:
            try:
                driver.quit()
            except:
                pass

# メイン処理
print(f"🚀 チェック開始: {len(accounts)} 件")
print(f"⏱️ 開始時刻: {datetime.now().strftime('%H:%M:%S')}")
print("="*50)

check_results = []
frozen_count = 0

# 並列処理（3件同時）
CONCURRENCY = 3
with concurrent.futures.ThreadPoolExecutor(max_workers=CONCURRENCY) as executor:
    futures = {executor.submit(check_account, acc): acc for acc in accounts}
    
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        result = future.result()
        check_results.append(result)
        
        row, account, status, reason = result
        if status in ['frozen', 'not_exist']:
            frozen_count += 1
            print(f"❄️ {account}: {reason}")
        
        # 進捗表示（10件ごと）
        if (i + 1) % 10 == 0 or i == len(accounts) - 1:
            pct = (i + 1) / len(accounts) * 100
            print(f"⏳ 進捗: {i + 1}/{len(accounts)} ({pct:.0f}%) - 凍結: {frozen_count}件")

print("="*50)
print(f"✅ チェック完了！")
print(f"⏱️ 終了時刻: {datetime.now().strftime('%H:%M:%S')}")
print(f"❄️ 凍結/存在しない: {frozen_count} 件")

In [ ]:
#@title 📝 結果をスプレッドシートに書き込み { display-mode: "form" }
from datetime import datetime

print("📝 結果をスプレッドシートに書き込み中...")

# 色の定義
COLOR_FROZEN = {'red': 1, 'green': 0.8, 'blue': 0.8}  # 薄い赤
COLOR_NOT_EXIST = {'red': 0.9, 'green': 0.9, 'blue': 0.9}  # グレー

# バッチ更新用のリクエストを作成
requests = []
col_idx = ord(ACCOUNT_COLUMN.upper()) - ord('A')

for row, account, status, reason in check_results:
    if status == 'frozen':
        color = COLOR_FROZEN
    elif status == 'not_exist':
        color = COLOR_NOT_EXIST
    else:
        continue  # 正常なアカウントはスキップ
    
    # 背景色を設定
    requests.append({
        'repeatCell': {
            'range': {
                'sheetId': worksheet.id,
                'startRowIndex': row - 1,
                'endRowIndex': row,
                'startColumnIndex': col_idx,
                'endColumnIndex': col_idx + 1
            },
            'cell': {
                'userEnteredFormat': {
                    'backgroundColor': color
                },
                'note': f"{reason} ({datetime.now().strftime('%Y/%m/%d %H:%M')})"
            },
            'fields': 'userEnteredFormat.backgroundColor,note'
        }
    })

# 一括更新
if requests:
    spreadsheet.batch_update({'requests': requests})
    print(f"✅ {len(requests)} 件のセルを更新しました")
else:
    print("ℹ️ 凍結アカウントはありませんでした")

print("")
print("🎉 完了！スプレッドシートを確認してください。")
print(f"📊 {SPREADSHEET_URL}")

---

## 📖 よくある質問

### Q: エラーが出た場合は？
「ランタイム」→「ランタイムを再起動」してからもう一度実行してください。

### Q: 90分経つとどうなる？
Colabは90分で自動的に停止します。再度このページを開いて「すべて実行」してください。

### Q: 色の意味は？
- 🔴 薄い赤: 凍結されているアカウント
- ⚪ グレー: 存在しないアカウント
- 色なし: 正常なアカウント